In [4]:
#!pip install pandas pandas-ta plotly ZigZag scipy TA-Lib tensorflow scikit-learn tqdm

Defaulting to user installation because normal site-packages is not writeable
  Using cached scikit_learn-1.1.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (30.4 MB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)


In [6]:
import tensorflow as tf
import os

import talib
import numpy as np
import pandas as pd
import pandas_ta as ta
from sklearn.preprocessing import scale
from tqdm import tqdm
import random

# Переменные

In [417]:
NAME = 'Take profit and stop loss prediction'
TIME_INTERVAL = 900
TAKE_STEPS_TO_WATCH = 64
INPUT_WIDTH = 512
LSTM_INPUT_WIDTH = 512
TAKE_PROFIT_PERCENT = 3
STOP_LOSS_PERCENT = 1
CURRENCY_PAIR = 'btc_usdt'
DROPOUT = 0.01        # 0.1
L2_REGULARIZERS = 0.002 # 0.02

In [418]:
LOOKAHEAD = False

In [419]:
os.getcwd()

'/mnt/d/Users/Neon/PycharmProjects/tradingBot/notebooks'

In [420]:
RATES_PATH = f"{os.getcwd()}/../csv"
EXAMPLE_PATH = f"{os.getcwd()}/checkpoints/"

if LOOKAHEAD:
  dataleak = 'DATA LEAK '
else:
  dataleak = ''

CHECKPOINT_PATH = f"{EXAMPLE_PATH}/{dataleak}{NAME} {CURRENCY_PAIR.upper()} {TIME_INTERVAL} steps {TAKE_STEPS_TO_WATCH} iw {INPUT_WIDTH} lstm {LSTM_INPUT_WIDTH} tp {TAKE_PROFIT_PERCENT} sl {STOP_LOSS_PERCENT}/"

if not os.path.exists(EXAMPLE_PATH):
  os.mkdir(EXAMPLE_PATH)

if not os.path.exists(CHECKPOINT_PATH):
  os.mkdir(CHECKPOINT_PATH)

# Функция индикаторов

In [421]:
#@title
def append_all_indicators(df):

  df = append_candles_indicators(df)
  df = append_overlap_indicators(df)
  df = append_momentum_indicators(df)
  df = append_statistics_indicators(df)
  df = append_trend_indicators(df)
  df = append_aberration_indicators(df)
  df = append_volume_indicators(df)
  
  return df

def append_candles_indicators(df):

  names = [
           "2crows",
          "3blackcrows",
          "3inside",
          "3linestrike",
          "3outside",
          "3starsinsouth",
          "3whitesoldiers",
          "abandonedbaby",
          "advanceblock",
          "belthold",
          "breakaway",
          "closingmarubozu",
          "concealbabyswall",
          "counterattack",
          "darkcloudcover",
          "doji",
          "dojistar",
          "dragonflydoji",
          "engulfing",
          "eveningdojistar",
          "eveningstar",
          "gapsidesidewhite",
          "gravestonedoji",
          "hammer",
          "hangingman",
          "harami",
          "haramicross",
          "highwave",
          "hikkake",
          "hikkakemod",
          "homingpigeon",
          "identical3crows",
          "inneck",
          "inside",
          "invertedhammer",
          "kicking",
          "kickingbylength",
          "ladderbottom",
          "longleggeddoji",
          "longline",
          "marubozu",
          "matchinglow",
          "mathold",
          "morningdojistar",
          "morningstar",
          "onneck",
          "piercing",
          "rickshawman",
          "risefall3methods",
          "separatinglines",
          "shootingstar",
          "shortline",
          "spinningtop",
          "stalledpattern",
          "sticksandwich",
          "takuri",
          "tasukigap",
          "thrusting",
          "tristar",
          "unique3river",
          "upsidegap2crows",
          "xsidegap3methods"
  ]

  df.ta.cdl_pattern(name=names, append=True)

  return df

def append_momentum_indicators(df):

  df.ta.apo(append=True)
  df.ta.bias(append=True)
  df.ta.bop(append=True)
  df.ta.brar(append=True)
  df.ta.cci(append=True)
  df.ta.cfo(append=True)
  df.ta.cg(append=True)
  df.ta.cmo(append=True)
  df.ta.coppock(append=True)
  df.ta.cti(append=True)
  df.ta.dm(append=True)
  df.ta.er(append=True)
  df.ta.eri(append=True)
  df.ta.fisher(append=True)
  df.ta.inertia(append=True)
  df.ta.kdj(append=True)
  df.ta.kst(append=True)
  df.ta.macd(append=True, )
  df.ta.mom(append=True)
  df.ta.pgo(append=True)
  df.ta.ppo(append=True)
  df.ta.psl(append=True)
  df.ta.pvo(append=True)
  df.ta.qqe(append=True)
  df.ta.roc(append=True)
  df.ta.rsi(append=True)
  df.ta.rsx(append=True)
  df.ta.rvgi(append=True)
  # df.ta.stc(append=True) вываливается ошибка
  df.ta.slope(append=True)
  df.ta.smi(append=True)
  df.ta.squeeze(append=True) 
  df.ta.squeeze_pro(append=True) 
  df.ta.stoch(append=True)
  df.ta.stochrsi(append=True)
  #df.ta.td_seq(append=True)   # медленный
  df['TD_SEQ_UP'] = 0
  df['TD_SEQ_DN'] = 0
  df.ta.trix(append=True)
  df.ta.tsi(append=True)
  df.ta.uo(append=True)
  df.ta.willr(append=True)

  return df

def append_overlap_indicators(df):

  df.ta.alma(append=True)
  df.ta.dema(append=True)
  df.ta.ema(append=True)
  df.ta.fwma(append=True)
  df.ta.hilo(append=True)
  df.ta.hl2(append=True)
  df.ta.hlc3(append=True)
  df.ta.hma(append=True)
  df.ta.hwma(append=True)
  df.ta.ichimoku(append=True, lookahead=LOOKAHEAD) # data leak
  #df.ta.jma(append=True)
  df.ta.kama(append=True)
  df.ta.linreg(append=True)
  df.ta.mcgd(append=True)
  df.ta.midpoint(append=True)
  df.ta.midprice(append=True)
  df.ta.ohlc4(append=True)
  df.ta.pwma(append=True)
  df.ta.rma(append=True)
  df.ta.sinwma(append=True)
  df.ta.sma(append=True)
  df.ta.ssf(append=True)
  df.ta.supertrend(append=True)
  df.ta.swma(append=True)
  df.ta.t3(append=True)
  df.ta.tema(append=True)
  df.ta.trima(append=True)
  df.ta.vidya(append=True)
  #df.ta.vwap(append=True) ошибка
  df.ta.vwma(append=True)
  df.ta.wcp(append=True) 
  df.ta.wma(append=True) 
  df.ta.zlma(append=True)

  return df

def append_statistics_indicators(df):
  df.ta.entropy(append=True)
  df.ta.kurtosis(append=True)
  df.ta.mad(append=True)
  df.ta.median(append=True)
  df.ta.quantile(append=True)
  df.ta.skew(append=True)
  df.ta.stdev(append=True)

  if LOOKAHEAD:
    df.ta.tos_stdevall(append=True) #возможная утечка данных в будущее
  else:
    df['TOS_STDEVALL_LR'] = 0
    df['TOS_STDEVALL_L_1'] = 0
    df['TOS_STDEVALL_U_1'] = 0
    df['TOS_STDEVALL_L_2'] = 0
    df['TOS_STDEVALL_U_2'] = 0
    df['TOS_STDEVALL_L_3'] = 0
    df['TOS_STDEVALL_U_3'] = 0

  df.ta.variance(append=True)
  df.ta.zscore(append=True)

  return df

def append_trend_indicators(df):
  df.ta.adx(append=True)
  df.ta.amat(append=True)
  df.ta.aroon(append=True)
  df.ta.chop(append=True)
  df.ta.cksp(append=True)
  #df.ta.decay(append=True) ошибка
  df.ta.decreasing(append=True)
  df.ta.dpo(append=True, lookahead=LOOKAHEAD) # data leak
  df.ta.increasing(append=True)
  df.ta.long_run(append=True)
  df.ta.psar(append=True)
  df.ta.qstick(append=True)
  df.ta.short_run(append=True)
  df.ta.tsignals(append=True)
  df.ta.ttm_trend(append=True)
  df.ta.vhf(append=True)
  df.ta.vortex(append=True)
  df.ta.xsignals(append=True)

  return df

def append_utility_indicators(df):
  # не работают
  df.ta.above(append=True)
  df.ta.above_value(append=True)
  df.ta.below(append=True)
  df.ta.below_value(append=True)
  df.ta.cross(append=True)

  return df

def append_aberration_indicators(df):
  df.ta.aberration(append=True)
  df.ta.accbands(append=True)
  df.ta.atr(append=True)
  df.ta.bbands(append=True)
  df.ta.donchian(append=True)
  #df.ta.hwc(append=True) ошибка
  df.ta.kc(append=True)
  df.ta.massi(append=True)
  df.ta.natr(append=True)
  df.ta.pdist(append=True)
  df.ta.rvi(append=True)
  df.ta.thermo(append=True)
  df.ta.true_range(append=True)
  df.ta.ui(append=True)

  return df

def append_volume_indicators(df):
  df.ta.ad(append=True)
  df.ta.adosc(append=True)
  df.ta.aobv(append=True)
  df.ta.cmf(append=True)
  df.ta.efi(append=True)
  df.ta.eom(append=True)
  df.ta.kvo(append=True)
  df.ta.mfi(append=True)
  df.ta.nvi(append=True)
  df.ta.obv(append=True)
  df.ta.pvi(append=True)
  df.ta.pvol(append=True)
  df.ta.pvr(append=True)
  df.ta.pvt(append=True)
  df.ta.vp(append=True)

  return df

def append_cycles(df):
  #df.ta.ebsw(append=True) появляется ошибка 

  return df

In [422]:
def append_indicators(df):

  #df.ta.macd(append=True, fast=12, slow=26, signal=9)
  #df.ta.macd(append=True, fast=8, slow=17, signal=9)
  #df.ta.ema(append=True, length=200)
  #df.ta.stoch(append=True, k=14, d=3)
  #df.ta.rsi(append=True, length=14)

  df = append_all_indicators(df)

  #for length in range(5, 200):
  #  df.ta.ema(append=True, length=length)

  return df

In [423]:
file_name = f'{CURRENCY_PAIR}_{TIME_INTERVAL}.csv.zip'

file_path = f'file://{RATES_PATH}/{file_name}'

df = pd.read_csv(file_path, decimal='.', keep_default_na=False, encoding = "UTF-8", compression='zip')

df = df.fillna(0)

df = (df.replace((np.inf, -np.inf), np.nan)
                     .dropna())

In [424]:
df

,datetime,open,close,low,high,volume
0,2017-10-12 08:30:00,5077.574899,5077.574899,5077.574899,5077.574899,0.001190
1,2017-10-12 08:45:00,5070.978000,5070.978000,5070.978000,5070.978000,0.001190
2,2017-10-12 09:00:00,5070.978000,5070.978000,5070.978000,5070.978000,0.000000
3,2017-10-12 09:15:00,5070.978000,5070.978000,5070.978000,5070.978000,0.000000
4,2017-10-12 09:30:00,5070.978000,5070.978000,5070.978000,5070.978000,0.000000
...,...,...,...,...,...,...
161696,2022-05-23 16:30:00,30422.930000,30402.170000,30350.010000,30454.550000,26.364489
161697,2022-05-23 16:45:00,30408.180000,30264.590000,30162.090000,30408.180000,30.137711
161698,2022-05-23 17:00:00,30264.590000,30146.210000,30043.810000,30265.400000,31.911468
161699,2022-05-23 17:15:00,30135.980000,30095.590000,30093.630000,30136.470000,0.314766


In [425]:
def processing(df, row, take_steps_to_watch):

  index = row.name

  loc = df.index.get_loc(index) + 1

  sample = df.iloc[loc:loc + take_steps_to_watch]

  position = 0

  for index, sample_row in sample.iterrows():
    if sample_row['high'] >= row['long_take_profit_price']:
      return 1
    if sample_row['low'] <= row['long_stop_loss_price']:
      return 0
      
  return position

def append_position(original_df):

  df = original_df.copy()

  df['long_take_profit_price'] = (100 + TAKE_PROFIT_PERCENT) * df['close'] / 100
  df['long_stop_loss_price'] = (100 - STOP_LOSS_PERCENT) * df['close'] / 100

  for index, row in df.iterrows():
    df.at[index, 'position'] = processing(df, row, TAKE_STEPS_TO_WATCH)

  original_df['position'] = df['position']

  return original_df

df = append_position(df)

In [426]:
def append_position(original_df):

  df = original_df.copy()

  df[f'future_high_{TAKE_STEPS_TO_WATCH}'] = df['high'].rolling(TAKE_STEPS_TO_WATCH).max().shift(-TAKE_STEPS_TO_WATCH)
  df[f'future_low_{TAKE_STEPS_TO_WATCH}'] = df['low'].rolling(TAKE_STEPS_TO_WATCH).min().shift(-TAKE_STEPS_TO_WATCH)

  df[f'future_high_{TAKE_STEPS_TO_WATCH}_percentage'] = (df[f'future_high_{TAKE_STEPS_TO_WATCH}'] * 100 / df['close']) - 100
  df[f'future_low_{TAKE_STEPS_TO_WATCH}_percentage'] = 100 - (df[f'future_low_{TAKE_STEPS_TO_WATCH}'] * 100 / df['close'])

  df['long_position'] = 0

  df.loc[(df[f'future_high_{TAKE_STEPS_TO_WATCH}_percentage'] > TAKE_PROFIT_PERCENT) & (df[f'future_low_{TAKE_STEPS_TO_WATCH}_percentage'] < STOP_LOSS_PERCENT), 'long_position'] = 1

  df['short_position'] = 0

  df.loc[(df[f'future_high_{TAKE_STEPS_TO_WATCH}_percentage'] < STOP_LOSS_PERCENT) & (df[f'future_low_{TAKE_STEPS_TO_WATCH}_percentage'] > TAKE_PROFIT_PERCENT), 'short_position'] = 1

  df['position'] = 0

  df.loc[df['long_position'] > 0, 'position'] = 1
  df.loc[df['short_position'] > 0, 'position'] = -1

  original_df['position'] = df['position']

  return original_df

#df = append_position(df)

In [427]:
print("Long trend count:", len(df[df['position'] > 0]))
print("Neutral trend count:", len(df[df['position'] == 0]))
print("Short trend count:", len(df[df['position'] < 0]))

Long trend count: 26959
Neutral trend count: 134742
Short trend count: 0


In [428]:
df = append_indicators(df)

In [429]:
index = df.columns

features_names_with_position = index.delete([
                                             index.get_loc('volume'), 
                                             index.get_loc('close'),
                                             index.get_loc('open'), 
                                             index.get_loc('high'), 
                                             index.get_loc('low'),
                                             ])

FEATURES_NAMES = features_names_with_position.delete([features_names_with_position.get_loc('position')]).values

FEATURES_NAMES

array(['datetime', 'CDL_2CROWS', 'CDL_3BLACKCROWS', 'CDL_3INSIDE',
       'CDL_3LINESTRIKE', 'CDL_3OUTSIDE', 'CDL_3STARSINSOUTH',
       'CDL_3WHITESOLDIERS', 'CDL_ABANDONEDBABY', 'CDL_ADVANCEBLOCK',
       'CDL_BELTHOLD', 'CDL_BREAKAWAY', 'CDL_CLOSINGMARUBOZU',
       'CDL_CONCEALBABYSWALL', 'CDL_COUNTERATTACK', 'CDL_DARKCLOUDCOVER',
       'CDL_DOJI_10_0.1', 'CDL_DOJISTAR', 'CDL_DRAGONFLYDOJI',
       'CDL_ENGULFING', 'CDL_EVENINGDOJISTAR', 'CDL_EVENINGSTAR',
       'CDL_GAPSIDESIDEWHITE', 'CDL_GRAVESTONEDOJI', 'CDL_HAMMER',
       'CDL_HANGINGMAN', 'CDL_HARAMI', 'CDL_HARAMICROSS', 'CDL_HIGHWAVE',
       'CDL_HIKKAKE', 'CDL_HIKKAKEMOD', 'CDL_HOMINGPIGEON',
       'CDL_IDENTICAL3CROWS', 'CDL_INNECK', 'CDL_INSIDE',
       'CDL_INVERTEDHAMMER', 'CDL_KICKING', 'CDL_KICKINGBYLENGTH',
       'CDL_LADDERBOTTOM', 'CDL_LONGLEGGEDDOJI', 'CDL_LONGLINE',
       'CDL_MARUBOZU', 'CDL_MATCHINGLOW', 'CDL_MATHOLD',
       'CDL_MORNINGDOJISTAR', 'CDL_MORNINGSTAR', 'CDL_ONNECK',
       'CDL_PIERCING', 

In [430]:
data = df.loc[:,features_names_with_position]

In [431]:
n = len(data)
max_val_df_length = 10000
max_eval_df_length = 10000
val_length = int(n*0.9) - int(n*0.8)
eval_length = int(n*1) - int(n*0.9)

if val_length > max_val_df_length:
    val_length = max_val_df_length
    
if eval_length > max_eval_df_length:
    eval_length = max_eval_df_length

val_start_index = n - eval_length - val_length
test_start_index = n - eval_length

In [432]:
print(n, val_start_index, test_start_index)

161701 141701 151701


In [433]:
train_df = data[0:val_start_index]
val_df = data[val_start_index:test_start_index]
test_df = data[test_start_index:]

test_full_df = df[test_start_index:]

num_features = data.shape[1]

In [434]:
print(len(data), len(train_df), len(val_df), len(test_df))

161701 141701 10000 10000


In [435]:
def normalization(dataframe, train_mean, train_std):
  position = dataframe.pop('position')
  dataframe = (dataframe - train_mean) / train_std
  dataframe['position'] = position
  return dataframe

In [436]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = normalization(train_df, train_mean, train_std)
val_df = normalization(val_df, train_mean, train_std)
test_df = normalization(test_df, train_mean, train_std)

/tmp/ipykernel_16806/3095889046.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_mean = train_df.mean()
/tmp/ipykernel_16806/3095889046.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_std = train_df.std()


In [437]:
print(train_mean, train_std)

position                0.169392
CDL_2CROWS             -0.032463
CDL_3BLACKCROWS        -0.037403
CDL_3INSIDE             0.109385
CDL_3LINESTRIKE         0.019054
                       ...      
mean_close          19756.277849
high_close          29247.932000
pos_volume         139063.407772
neg_volume         140297.129487
total_volume       279360.537259
Length: 265, dtype: float64 position                0.375099
CDL_2CROWS              1.801455
CDL_3BLACKCROWS         1.933623
CDL_3INSIDE            11.115708
CDL_3LINESTRIKE         4.803807
                       ...      
mean_close          17903.194580
high_close          24071.801013
pos_volume          74785.481637
neg_volume          74934.125837
total_volume       149602.986716
Length: 265, dtype: float64


In [438]:
print(len(train_df), len(val_df), len(test_df))

141701 10000 10000


In [439]:
train_df = train_df.fillna(0)
val_df = val_df.fillna(0)
test_df = test_df.fillna(0)

train_df[train_df.isnull().any(axis=1)]
train_df[train_df.isna().any(axis=1)]

,ABER_ATR_5_15,ABER_SG_5_15,ABER_XG_5_15,ABER_ZG_5_15,ACCBL_20,ACCBM_20,ACCBU_20,AD,ADOSC_3_10,ADX_14,...,ZL_EMA_10,ZS_30,datetime,high_close,low_close,mean_close,neg_volume,pos_volume,position,total_volume


In [440]:
assert train_df.isnull().values.any() == False, 'Тренировочный датафрейм не должен содержать nan значений'
assert val_df.isnull().values.any() == False, 'Датафрейм валидации не должен содержать nan значений'
assert test_df.isnull().values.any() == False, 'Тестовый датафрейм не должен содержать nan значений'

In [441]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
               label_columns=None, input_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    self.input_columns = input_columns

    self.input_columns_indices = {name: i for i, name in
                                    enumerate(input_columns)}   

    self.label_columns = label_columns

    self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}                       

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Column indices: {self.column_indices}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}',
        f'Label start: {self.label_start}',
        f'Input width: {self.input_width}',
        f'Label width: {self.label_width}'])

In [442]:
def make_dataset(self, data, shuffle=True, batch_size=32,):
  data = np.array(data, dtype=np.float32)

  ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=shuffle,
      batch_size=batch_size)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [443]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [444]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]

  inputs = tf.stack(
        [inputs[:, :, self.column_indices[name]] for name in self.input_columns],
        axis=-1)
  
  labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [445]:
class Model():
  def __init__(self, model, path, max_epochs=30, patience=3):
    self.model = model
    self.path = path
    self.max_epochs = max_epochs
    self.patience = patience

  def compile(self):
    self.model.compile(loss=tf.losses.MeanSquaredError(),
                  optimizer=tf.optimizers.Adam(learning_rate=0.0005),
                  metrics=[tf.metrics.MeanAbsoluteError()])

  def load_weights(self):
    self.model.load_weights(self.path)

  def fit(self, window):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=self.patience,
                                                      mode='min')

    # Create a callback that saves the model's weights
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=self.path,
                                                    save_weights_only=True,
                                                    verbose=1)

    history = self.model.fit(window.train, epochs=self.max_epochs,
                        validation_data=window.val,
                        callbacks=[early_stopping, cp_callback])
    
    return history

In [446]:
wide_window = WindowGenerator(
    train_df=train_df, val_df=val_df, test_df=test_df,
    input_width=INPUT_WIDTH, label_width=1, shift=1,
    label_columns=['position'], input_columns=FEATURES_NAMES)

In [447]:
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(LSTM_INPUT_WIDTH, activation='tanh', return_sequences=False, kernel_regularizer=tf.keras.regularizers.l2(L2_REGULARIZERS)),
    tf.keras.layers.Dropout(DROPOUT),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(1)
])

# Тренировка нейросети

In [448]:
model = Model(lstm_model, CHECKPOINT_PATH, max_epochs=20, patience=3)
model.compile()

if os.path.exists(f'{CHECKPOINT_PATH}/checkpoint'):
  model.load_weights()

In [449]:
model.fit(wide_window)

Epoch 1/20
4413/4413 [==============================] - ETA: 0s - loss: 0.2022 - mean_absolute_error: 0.2769
Epoch 1: saving model to /mnt/d/Users/Neon/PycharmProjects/tradingBot/notebooks/checkpoints//Take profit and stop loss prediction BTC_USDT 900 steps 64 iw 512 lstm 512 tp 3 sl 1/
4413/4413 [==============================] - 10020s 2s/step - loss: 0.2022 - mean_absolute_error: 0.2769 - val_loss: 0.1319 - val_mean_absolute_error: 0.2895
Epoch 2/20
4413/4413 [==============================] - ETA: 0s - loss: 0.1373 - mean_absolute_error: 0.2689
Epoch 2: saving model to /mnt/d/Users/Neon/PycharmProjects/tradingBot/notebooks/checkpoints//Take profit and stop loss prediction BTC_USDT 900 steps 64 iw 512 lstm 512 tp 3 sl 1/
4413/4413 [==============================] - 10026s 2s/step - loss: 0.1373 - mean_absolute_error: 0.2689 - val_loss: 0.1322 - val_mean_absolute_error: 0.2933
Epoch 3/20
4413/4413 [==============================] - ETA: 0s - loss: 0.1357 - mean_absolute_error: 0.2641

In [450]:
@property
def test_without_shuffle(self):

  return self.make_dataset(self.test_df, shuffle=False)

WindowGenerator.test_without_shuffle = test_without_shuffle

In [451]:
model = lstm_model

In [452]:
model.evaluate(wide_window.test)

297/297 [==============================] - 251s 842ms/step - loss: 0.1307 - mean_absolute_error: 0.2768


[0.1306837648153305, 0.27681708335876465]

In [453]:
dataset = wide_window.test_without_shuffle.map(lambda x, y: model(x)).unbatch()

In [454]:
index = test_full_df.iloc[0].name

before = np.full(INPUT_WIDTH, 0.0)
after = np.fromiter(dataset.as_numpy_iterator(), np.float32)

position_prediction_df = pd.DataFrame(data=np.concatenate((before, after), axis=0), columns=['position'], index=test_full_df.index)

In [455]:
dataframe_with_prediction = pd.concat([position_prediction_df, test_full_df.drop(columns=['position'])], axis=1, join="inner")

In [456]:
#@title
from pandas import DataFrame
class Strategy:
    fee = 0.2
    # ограничение на максимальное количество открытых сделок
    max_simultaneous_open_trades_limit = 1
    # максимальное количество сделок которое было открыто одновременно
    max_simultaneous_open_trades_count = 0

    def __init__(self, max_simultaneous_open_trades_limit=1, start_balance=1000):
        self.orders = pd.DataFrame(None,
                                   columns=['type', 'open_price', 'open_datetime', 'close_price', 'close_datetime',
                                            'profit', 'quantity', 'profit_as_a_percentage', 'take_profit_percentage',
                                            'stop_loss_percentage', 'take_profit_price', 'stop_loss_price'],
                                   ).astype({
            'type': str, 'open_price': np.float32, 'open_datetime': str, 'close_price': np.float32,
            'close_datetime': str,
            'profit': np.float32, 'quantity': np.float32,
            'profit_as_a_percentage': np.float32, 'take_profit_percentage': np.float32,
            'stop_loss_percentage': np.float32, 'take_profit_price': np.float32, 'stop_loss_price': np.float32
        })

        self.max_simultaneous_open_trades_limit = max_simultaneous_open_trades_limit
        self.start_balance = start_balance
        self.balance = start_balance

    def append_indicators(self, df: DataFrame) -> DataFrame:
        return df

    def get_orders(self):
        return self.orders

    def put_order(self, type: str, price: float, quantity: float, datetime: str = None,
                  close_price: float = None, close_datetime: str = None,
                  take_profit_percentage: float = None,
                  stop_loss_percentage: float = None) -> int:

        row = {
            'type': type,
            'open_price': price,
            'open_datetime': datetime,
            'close_price': close_price,
            'close_datetime': close_datetime,
            'profit': None,
            'quantity': quantity,
            'profit_as_a_percentage': None,
            'take_profit_percentage': take_profit_percentage,
            'stop_loss_percentage': stop_loss_percentage,
            'take_profit_price': None,
            'stop_loss_price': None
        }

        if take_profit_percentage is not None:
            row['take_profit_price'] = (row['take_profit_percentage'] + 100) * price / 100

        if stop_loss_percentage is not None:
            row['stop_loss_price'] = (100 - row['stop_loss_percentage']) * price / 100

        self.orders.loc[self.orders.shape[0]] = row

        open_trades_count = len(self.get_open_orders())

        if open_trades_count > self.max_simultaneous_open_trades_count:
            self.max_simultaneous_open_trades_count = open_trades_count

        self.balance = self.balance - quantity - quantity * self.fee / 100

        return self.orders.iloc[-1].name

    def is_order_closed(self, index: int):
        if self.orders.at[index, 'close_price'] or self.orders.at[index, 'close_datetime']:
            return True
        else:
            return False

    def close_order(self, index: int, price: float, datetime: str) -> bool:

        index = int(index)

        if self.is_order_closed(index):
            return False

        self.orders.at[index, 'close_price'] = float(price)
        self.orders.at[index, 'close_datetime'] = str(datetime)

        order = self.orders.iloc[index]

        fee_for_opening_trade = order['quantity'] * self.fee / 100
        fee_for_closing_trade = order['quantity'] * self.fee / 100

        if order['type'] == 'long':

            profit_as_a_percentage_without_commission = order['close_price'] * 100 / order['open_price'] - 100

            amount_on_close_trade = (order['quantity'] * (100 + profit_as_a_percentage_without_commission) / 100)

            self.balance += amount_on_close_trade - fee_for_closing_trade

            self.orders.at[index, 'profit'] = amount_on_close_trade - order['quantity'] - fee_for_opening_trade - fee_for_closing_trade
            self.orders.at[index, 'profit_as_a_percentage'] = profit_as_a_percentage_without_commission - self.fee * 2

        elif order['type'] == 'short':

            profit_as_a_percentage_without_commission = 100 - order['close_price'] * 100 / order['open_price']

            amount_on_close_trade = (order['quantity'] * (100 + profit_as_a_percentage_without_commission) / 100)

            self.balance += amount_on_close_trade - fee_for_closing_trade

            self.orders.at[index, 'profit'] = amount_on_close_trade - order['quantity'] - fee_for_opening_trade \
                                              - fee_for_closing_trade
            self.orders.at[index, 'profit_as_a_percentage'] = profit_as_a_percentage_without_commission - self.fee * 2

        return True

    def get_open_orders(self) -> DataFrame:
        return self.orders[(self.orders['close_price'].isna()) & (pd.notna(self.orders['open_datetime']))]

    def get_closed_orders(self) -> DataFrame:
        return self.orders[pd.notna(self.orders['close_price'])]

    def get_profit(self) -> float:
        return self.orders['profit'].sum()

    def get_profit_as_a_percentage(self) -> float:
        return self.orders['profit_as_a_percentage'].sum()

    def get_opened_orders_count(self) -> int:
        return len(self.get_open_orders())

    def get_closed_orders_count(self) -> int:
        return len(self.get_closed_orders())

    def get_win_trades(self) -> DataFrame:
        closed = self.get_closed_orders()
        return closed[closed['profit_as_a_percentage'] > 0]

    def get_loss_trades(self) -> DataFrame:
        closed = self.get_closed_orders()
        return closed[closed['profit_as_a_percentage'] < 0]

    def get_win_ration(self) -> float:
        return 100 * len(self.get_win_trades()) / self.get_trades_count()

    def get_trades_count(self) -> int:
        return self.get_closed_orders_count()

    def has_opened_orders(self) -> bool:
        return len(self.get_open_orders()) > 0

    def get_max_simultaneous_open_trades_count(self):
        return self.max_simultaneous_open_trades_count

    def get_pending_orders(self):
        return self.orders[(self.orders['open_datetime'].isna()) & (pd.notna(self.orders['open_price']))]

    def tick(self, candlestick):

        open_trades = self.get_open_orders()

        if len(open_trades) > 0:

            for index, item in open_trades[candlestick['low'] < open_trades['stop_loss_price']].iterrows():
                self.close_order(index, item['stop_loss_price'], candlestick['datetime'])

            for index, item in open_trades[candlestick['high'] > open_trades['take_profit_price']].iterrows():
                self.close_order(index, item['take_profit_price'], candlestick['datetime'])

        pending_orders = self.get_pending_orders()

        if len(pending_orders) > 0:

            long_pending_orders = pending_orders[pending_orders['type'] == 'long']

            for index, item in long_pending_orders[(candlestick['low'] < long_pending_orders['open_price']) & (
                    candlestick['high'] > long_pending_orders['open_price'])].iterrows():
                self.open_order(index, candlestick['datetime'])

    def open_order(self, index: int, open_datetime: np.str):
        self.orders.at[index, 'open_datetime'] = open_datetime

    def backtest(self, df: DataFrame):
        pbar = tqdm(df.iterrows(), total=len(df), colour='green')

        for index, candlestick in pbar:
            self.tick(candlestick=candlestick)
            pbar.set_description(
                f"Strategy backtesting {index} Balance: {self.get_available_balance()} Open trades: {len(self.get_open_orders())} ")

    def close_all_open_trades(self, candlestick):

        open_trades = self.get_open_orders()

        for index, item in open_trades.iterrows():
            self.close_order(index, candlestick['close'], candlestick['datetime'])

        pending_trades = self.get_pending_orders()

        for index, item in pending_trades.iterrows():
            self.cancel_order(index)

    def get_available_balance(self) -> float:
        return self.balance

    def get_start_balance(self) -> float:
        return self.start_balance

    def cancel_order(self, index: int) -> bool:

        index = int(index)

        self.orders.drop([index], inplace=True)

        return True


/tmp/ipykernel_16806/4095104747.py:179: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def open_order(self, index: int, open_datetime: np.str):


In [469]:
MAX_POSITION = dataframe_with_prediction['position'].max()
LIMIT = 80 * MAX_POSITION / 100

print(MAX_POSITION, LIMIT)

0.4342438876628876 0.34739511013031005


In [470]:
class LSTMStrategy(Strategy):
    fee = 0.1
    order_id = None

    def append_indicators(self, df: DataFrame) -> DataFrame:
        #df.ta.rsi(length=self.rsi, append=True)

        df = df.fillna(0)

        df = (df.replace((np.inf, -np.inf), np.nan).dropna())

        return df

    def tick(self, candlestick):
        super().tick(candlestick)

        if self.get_opened_orders_count() < 1:
          if candlestick['position'] > LIMIT:
              self.order_id = self.put_order(
                  type='long',
                  price=candlestick['close'],
                  datetime=candlestick['datetime'],
                  quantity=500,
                  take_profit_percentage=TAKE_PROFIT_PERCENT,
                  stop_loss_percentage=STOP_LOSS_PERCENT
              )

In [471]:
strategy = LSTMStrategy()

In [472]:
strategy.backtest(dataframe_with_prediction)
strategy.close_all_open_trades(dataframe_with_prediction.iloc[-1])

Strategy backtesting 161700 Balance: 988.0 Open trades: 0 : 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:34<00:00, 290.99it/s]


In [473]:
strategy.get_orders()

,type,open_price,open_datetime,close_price,close_datetime,profit,quantity,profit_as_a_percentage,take_profit_percentage,stop_loss_percentage,take_profit_price,stop_loss_price
0,long,44152.01,2022-03-01 22:30:00,43710.4899,2022-03-02 07:15:00,-6.0,500.0,-1.2,3.0,1.0,45476.5703,43710.4899
1,long,38461.65,2022-03-08 15:30:00,39615.4995,2022-03-09 03:15:00,14.0,500.0,2.8,3.0,1.0,39615.4995,38077.0335
2,long,41986.00,2022-03-19 00:30:00,41566.14,2022-03-19 03:45:00,-6.0,500.0,-1.2,3.0,1.0,43245.5800,41566.1400
3,long,44240.98,2022-03-25 17:30:00,45568.2094,2022-03-27 20:30:00,14.0,500.0,2.8,3.0,1.0,45568.2094,43798.5702
4,long,38523.61,2022-05-02 02:30:00,38138.3739,2022-05-02 18:00:00,-6.0,500.0,-1.2,3.0,1.0,39679.3183,38138.3739
5,long,28361.64,2022-05-12 18:45:00,28078.0236,2022-05-12 21:30:00,-6.0,500.0,-1.2,3.0,1.0,29212.4892,28078.0236
6,long,28436.57,2022-05-12 21:30:00,29289.6671,2022-05-13 00:30:00,14.0,500.0,2.8,3.0,1.0,29289.6671,28152.2043
7,long,30384.06,2022-05-13 06:45:00,30080.2194,2022-05-13 16:30:00,-6.0,500.0,-1.2,3.0,1.0,31295.5818,30080.2194
8,long,29905.59,2022-05-13 16:30:00,29606.5341,2022-05-13 17:45:00,-6.0,500.0,-1.2,3.0,1.0,30802.7577,29606.5341
9,long,29600.00,2022-05-13 17:45:00,29304.0,2022-05-13 23:30:00,-6.0,500.0,-1.2,3.0,1.0,30488.0000,29304.0000


# Результаты тестирования стратегии

In [474]:
if len(strategy.get_orders()):
  print(f'Start balance: {strategy.get_start_balance()}')
  print(f'Available balance: {strategy.get_available_balance()}')
  print(f'Profit as percentage: {strategy.get_profit_as_a_percentage()}')
  print(f'Profit: {strategy.get_profit()}')
  print(f'Win ratio: {strategy.get_win_ration()}')
  print(f'Max simultaneous trades count: {strategy.get_max_simultaneous_open_trades_count()}')

Start balance: 1000
Available balance: 988.0
Profit as percentage: -2.4000000000000004
Profit: -11.999999999999943
Win ratio: 25.0
Max simultaneous trades count: 1
